<a href="https://colab.research.google.com/github/Tristancarlisle99/HCML/blob/main/fullQLoRA%26RAG_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment Setup

In [ ]:
%%capture
!pip install peft
!pip install huggingface_hub
!pip install transformers
!pip install accelerate
!pip install bitsandbytes
!pip install -q -U trl
!pip install ninja

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from pathlib import Path
from typing import Optional
from trl import SFTTrainer
import os

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

# Down Stream task directory set up



In [ ]:
def Downstream_task_checkpoint_directories(path: Optional[Path]= None, dir_name: str="LoRAs"):
  current_dir = path if path is not None else Path('./')
  output_dir = current_dir / dir_name
  output_dir.mkdir(parents=True, exist_ok=True)
  return output_dir

In [ ]:
DeciLora = Downstream_task_checkpoint_directories(dir_name="DeciLora_finetuned_checkpoints")
print(f"Output directory created: {DeciLora}")

Output directory created: DeciLora_finetuned_checkpoints


# Quantization config - 4 bit and The matrix multiplication and training will be faster if one uses a 16-bit compute dtype





In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16)

# Model selection and set up

In [ ]:
model_id = 'Deci/DeciLM-7B'
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             device_map="auto",
                                             trust_remote_code=True,
                                             quantization_config=bnb_config
                                             )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/895 [00:00<?, ?B/s]

configuration_decilm.py:   0%|          | 0.00/576 [00:00<?, ?B/s]

version_check.py:   0%|          | 0.00/383 [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/Deci/DeciLM-7B:
- version_check.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer.pad_token = tokenizer.eos_token

tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

# Dataset loader

In [ ]:
from datasets import load_dataset

In [ ]:
PTBXL_fintuning_dataset_path =  '/content/drive/MyDrive/finetuningdatasets/finetuningECG.csv'
dataset = load_dataset("csv", data_files= PTBXL_fintuning_dataset_path, split="train")
data = dataset.shuffle(seed=42)
ECG_statements = data.train_test_split(test_size=0.1, seed=42)

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
def prompt_construction(sample,modelname='DeciLM'):
  if modelname == 'DeciLM':
    prompt = ""
    prompt += "Generate a technical ECG report based on these Standard Communication Protocol for Computer-assisted Electrocardiography codes (SCP codes) (12 lead) "
    prompt += sample["input"]
    prompt += "\n\n### Report: \n "
    prompt += sample["output"]
    return {"text" : prompt}


In [ ]:
ECG_statements_dataset = ECG_statements.map(prompt_construction)

Map:   0%|          | 0/19247 [00:00<?, ? examples/s]

Map:   0%|          | 0/2139 [00:00<?, ? examples/s]

In [ ]:
ECG_statements_dataset['train'][42]['text']

"Generate a technical ECG report based on these Standard Communication Protocol for Computer-assisted Electrocardiography codes (SCP codes) (12 lead) Description: ['left ventricular hypertrophy'] , SCP_codes: {'Superclass': {'NORM': 0, 'MI': 0, 'STTC': 0, 'CD': 0, 'HYP': 1}, 'Subclass': {'HYP': ['LVH'], 'MI': None, 'STTC': None, 'CD': None}}\n\n### Report: \n sinus rhythm. voltages are high in chest leads suggesting lvh. borderline left axis deviation. Edit: LVH 35, Sokolow 3.6 (LVH 50)"

# QLoRA configuration

Website for config params:
https://huggingface.co/docs/peft/package_reference/lora


Layers which we can apply LoRA to for training.

In [ ]:
print([(n, type(m)) for n, m in model.named_modules()])

[('', <class 'transformers_modules.Deci.DeciLM-7B.cdd1aef103a52ac458dee200ad0661c967dd0e63.modeling_decilm.DeciLMForCausalLM'>), ('model', <class 'transformers_modules.Deci.DeciLM-7B.cdd1aef103a52ac458dee200ad0661c967dd0e63.modeling_decilm.DeciLMModel'>), ('model.embed_tokens', <class 'torch.nn.modules.sparse.Embedding'>), ('model.layers', <class 'torch.nn.modules.container.ModuleList'>), ('model.layers.0', <class 'transformers_modules.Deci.DeciLM-7B.cdd1aef103a52ac458dee200ad0661c967dd0e63.modeling_decilm.DeciLMDecoderLayer'>), ('model.layers.0.self_attn', <class 'transformers_modules.Deci.DeciLM-7B.cdd1aef103a52ac458dee200ad0661c967dd0e63.modeling_decilm.DeciLMAttention'>), ('model.layers.0.self_attn.q_proj', <class 'bitsandbytes.nn.modules.Linear4bit'>), ('model.layers.0.self_attn.k_proj', <class 'bitsandbytes.nn.modules.Linear4bit'>), ('model.layers.0.self_attn.v_proj', <class 'bitsandbytes.nn.modules.Linear4bit'>), ('model.layers.0.self_attn.o_proj', <class 'bitsandbytes.nn.module

**MLP**
1.   up_proj
2.   gate_proj
3.   down_proj

**Attention**
1.   q_proj
2.   k_proj
3.   v_proj




In [ ]:
import peft

from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

lora_config = LoraConfig(
    r=256,
    lora_alpha=512,
    lora_dropout=0.1,
    target_modules = ["gate_proj", "down_proj", "up_proj","q_proj", "v_proj","k_proj",],
    task_type="CAUSAL_LM"
)

Prepping model for quyantization

In [ ]:
decilm = prepare_model_for_kbit_training(model)
decilm = get_peft_model(decilm, lora_config)

# Training config

In [ ]:
import transformers
from transformers import TrainingArguments

training_args = TrainingArguments(
        output_dir=DeciLora,
        evaluation_strategy="steps",
        do_eval=True,
        auto_find_batch_size=True,
        log_level="debug",
        optim="paged_adamw_8bit",
        save_steps=25,
        logging_steps=100,
        learning_rate=2e-4,
        weight_decay=0.01,
        max_steps=len(ECG_statements_dataset['train']),
        warmup_steps=150,
        fp16=True,
        gradient_checkpointing=True,
        max_grad_norm=0.3,
        lr_scheduler_type="reduce_lr_on_plateau",
)

In [ ]:
trainer = SFTTrainer(
    model=decilm,
    args=training_args,
    peft_config=lora_config,
    tokenizer=tokenizer,
    dataset_text_field='text',
    train_dataset=ECG_statements_dataset['train'],
    eval_dataset=ECG_statements_dataset['test'],
    max_seq_length=4096,
    dataset_num_proc=os.cpu_count(),
)

Map (num_proc=8):   0%|          | 0/19247 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/2139 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs
Using auto half precision backend


In [ ]:
trainer.train()

Currently training with a batch size of: 8
***** Running training *****
  Num examples = 19,247
  Num Epochs = 8
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 19,247
  Number of trainable parameters = 591,593,472
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two va

Step,Training Loss,Validation Loss
100,0.858500,0.392141
200,0.358600,0.292164
300,3.536800,3.249036
400,2.417900,1.945753
500,1.345500,0.961491
600,0.865100,0.915472
700,0.773000,0.698359
800,0.759300,0.560320
900,0.526100,nan


Saving model checkpoint to DeciLora_finetuned_checkpoints/checkpoint-25
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--Deci--DeciLM-7B/snapshots/cdd1aef103a52ac458dee200ad0661c967dd0e63/config.json
Model config DeciLMConfig {
  "architectures": [
    "DeciLMForCausalLM"
  ],
  "attention_bias": false,
  "auto_map": {
    "AutoConfig": "Deci/DeciLM-7B--configuration_decilm.DeciLMConfig",
    "AutoModelForCausalLM": "Deci/DeciLM-7B--modeling_decilm.DeciLMForCausalLM"
  },
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 8192,
  "model_type": "deci",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "num_key_value_heads_per_layer": [
    4,
    4,
    4,
    4,
    4,
    2,
    2,
    2,
    2,
    2,
    4,
    2,
    2,
    2,
    2,
    2,
    2,
    2,
    2,
    2,
    1,
    1,

SafetensorError: Error while serializing: IoError(Os { code: 28, kind: StorageFull, message: "No space left on device" })

In [ ]:
trainer.save_model()

#RAG

In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext

In [ ]:
documents = SimpleDirectoryReader("/content/RAG_documents/").load_data()

In [ ]:
import torch

from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt
llm = LlamaCPP(
    model_url='https://huggingface.co/TristanCarlisle/Mistral-7B-ECGfinetunedmodel.gguf',
    model_path=None,
    temperature=0.1,
    max_new_tokens=256,
    context_window=3900,
    generate_kwargs={},
    model_kwargs={"n_gpu_layers": -1},
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)

In [ ]:
!pip -q install sentence-transformers
!pip -q install langchain

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from llama_index import ServiceContext

embed_model = HuggingFaceEmbeddings(model_name="thenlper/gte-large")

In [ ]:
service_context = ServiceContext.from_defaults(
    chunk_size=256,
    llm=llm,
    embed_model=embed_model
)

In [ ]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)
index.storage_context.persist(persist_dir="./storage")
query_engine = index.as_query_engine(streaming=True, similarity_top_k=1, service_context=service_context)
response_stream = query_engine.query(f"Describe this term Left ventricular hypertophy and its ECG presentation in lead I")
response_stream.print_response_stream()